In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

import warnings
warnings.filterwarnings('ignore')

데이터 불러오기 

data1 : phenotype_1000

data2 : KCPS

data3 : phenotype_1000 + KCPS

In [36]:
data1=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/phenotype_1000.txt',engine="python",sep=" ")

In [37]:
data2=pd.read_csv('C:/Users/dudtj/OneDrive - 숭실대학교 - Soongsil University/Desktop/CL/python/동계인턴십_Data_1000/KCPS_r2_0.01_1000.csv')

In [38]:
for i in range(2,15190):
    data2.columns.values[i]=i

In [39]:
data1=data1.loc[:,['LUNG','FID','AGE_B', 'SMOK_B', 'ALCO_B','EXER_B','HT_B', 'WT_B', 'WAIST_B', 'SBP_B', 'DBP_B', 'CHO_B','LDL_B', 'TG_B', 'HDL_B', 'FBS_B', 'GOT_B', 'GPT_B', 'GGT_B', 'URIC_B','BIL', 'WBC', 'CREAT','SEX1', 'CRC']]

In [40]:
import matplotlib.pyplot as plt 
import seaborn as sns

In [41]:
data1 = pd.concat([data1,data2],axis=1)

In [42]:
data1.dropna(inplace=True)
data1.drop('FID',axis=1,inplace=True)
data1.drop('IID',axis=1,inplace=True)

In [9]:
#corr = data1.corr(method='pearson')
#colormap = plt.cm.PuBu 
#plt.figure(figsize=(15,15)) 
#plt.title("Person Correlation of Features", y = 1.05, size = 15) 
#sns.heatmap(data1.astype(float).corr(),fmt='.2f', linewidths = 0.1, vmax = 1.0, square = True, cmap = colormap, linecolor = "white", annot = True, annot_kws = {"size" : 10})

In [43]:
data1

,LUNG,AGE_B,SMOK_B,ALCO_B,EXER_B,HT_B,WT_B,WAIST_B,SBP_B,DBP_B,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,0,48,3.0,2.0,1.0,168.0,69.8,82.0,102.0,69.0,...,0,2,0,2,0,1,1,1,0,2
1,0,53,1.0,2.0,1.0,161.0,67.9,86.0,127.0,67.0,...,0,1,0,1,0,0,1,0,2,0
2,0,60,1.0,1.0,2.0,153.0,73.3,97.0,133.0,76.0,...,0,2,0,0,0,2,0,2,0,0
3,0,42,1.0,2.0,2.0,167.0,65.1,77.5,97.0,70.0,...,0,0,0,0,0,0,1,1,1,0
4,0,57,3.0,2.0,2.0,165.0,51.8,73.0,98.0,76.0,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,0,37,3.0,2.0,2.0,173.0,69.4,82.0,145.0,87.0,...,0,2,0,1,0,0,0,1,2,0
996,0,57,2.0,2.0,1.0,159.0,61.4,87.0,109.0,80.0,...,0,2,0,1,0,0,0,2,0,2
997,0,50,2.0,2.0,1.0,163.0,66.4,82.0,150.0,82.0,...,0,1,0,2,0,2,1,1,1,1
998,0,45,1.0,2.0,1.0,156.0,48.0,67.5,91.0,60.0,...,0,2,0,0,0,1,1,1,0,0


In [44]:
y_df = data1.iloc[:, 0]

In [45]:
data1.drop('LUNG',axis=1,inplace=True)
X_df = data1

In [46]:
X_df

,AGE_B,SMOK_B,ALCO_B,EXER_B,HT_B,WT_B,WAIST_B,SBP_B,DBP_B,CHO_B,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,48,3.0,2.0,1.0,168.0,69.8,82.0,102.0,69.0,196.0,...,0,2,0,2,0,1,1,1,0,2
1,53,1.0,2.0,1.0,161.0,67.9,86.0,127.0,67.0,159.0,...,0,1,0,1,0,0,1,0,2,0
2,60,1.0,1.0,2.0,153.0,73.3,97.0,133.0,76.0,198.0,...,0,2,0,0,0,2,0,2,0,0
3,42,1.0,2.0,2.0,167.0,65.1,77.5,97.0,70.0,139.0,...,0,0,0,0,0,0,1,1,1,0
4,57,3.0,2.0,2.0,165.0,51.8,73.0,98.0,76.0,166.0,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,37,3.0,2.0,2.0,173.0,69.4,82.0,145.0,87.0,156.0,...,0,2,0,1,0,0,0,1,2,0
996,57,2.0,2.0,1.0,159.0,61.4,87.0,109.0,80.0,194.0,...,0,2,0,1,0,0,0,2,0,2
997,50,2.0,2.0,1.0,163.0,66.4,82.0,150.0,82.0,210.0,...,0,1,0,2,0,2,1,1,1,1
998,45,1.0,2.0,1.0,156.0,48.0,67.5,91.0,60.0,159.0,...,0,2,0,0,0,1,1,1,0,0


In [14]:
#y_df =data1['LUNG']
#X_df =data1.drop('LUNG', axis=1)

In [47]:
import numpy as np
import  pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score # 평가를 위함
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler, Binarizer

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier

In [48]:
from sklearn.linear_model import Lasso

In [17]:
#X_df_columns=X_df.iloc[:,lasso_low.coef_!=0].columns

In [18]:
#X_df_columns

In [19]:
#X_df=data1.loc[:,X_df_columns]

In [49]:
X_df

,AGE_B,SMOK_B,ALCO_B,EXER_B,HT_B,WT_B,WAIST_B,SBP_B,DBP_B,CHO_B,...,15180,15181,15182,15183,15184,15185,15186,15187,15188,15189
0,48,3.0,2.0,1.0,168.0,69.8,82.0,102.0,69.0,196.0,...,0,2,0,2,0,1,1,1,0,2
1,53,1.0,2.0,1.0,161.0,67.9,86.0,127.0,67.0,159.0,...,0,1,0,1,0,0,1,0,2,0
2,60,1.0,1.0,2.0,153.0,73.3,97.0,133.0,76.0,198.0,...,0,2,0,0,0,2,0,2,0,0
3,42,1.0,2.0,2.0,167.0,65.1,77.5,97.0,70.0,139.0,...,0,0,0,0,0,0,1,1,1,0
4,57,3.0,2.0,2.0,165.0,51.8,73.0,98.0,76.0,166.0,...,0,2,0,2,0,1,0,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,37,3.0,2.0,2.0,173.0,69.4,82.0,145.0,87.0,156.0,...,0,2,0,1,0,0,0,1,2,0
996,57,2.0,2.0,1.0,159.0,61.4,87.0,109.0,80.0,194.0,...,0,2,0,1,0,0,0,2,0,2
997,50,2.0,2.0,1.0,163.0,66.4,82.0,150.0,82.0,210.0,...,0,1,0,2,0,2,1,1,1,1
998,45,1.0,2.0,1.0,156.0,48.0,67.5,91.0,60.0,159.0,...,0,2,0,0,0,1,1,1,0,0


In [50]:
# random_state = 156 : 다음에도 같은 데이터를 train, test로 분리 하는 옵션
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_df,y_df,test_size=0.2,random_state=156,stratify=y_df)

In [51]:
lasso = Lasso(alpha=0.001, max_iter=100000).fit(X_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.99
테스트 세트의 정확도 : -0.19
사용한 특성의 수 : 573
사용한 max_iter : 1004


In [54]:
lasso = Lasso(alpha=0.0105, max_iter=100000).fit(X_train, y_train)

print("훈련 세트의 정확도 : {:.2f}".format(lasso.score(X_train, y_train)))
print("테스트 세트의 정확도 : {:.2f}".format(lasso.score(X_test, y_test)))
print("사용한 특성의 수 : {}".format(np.sum(lasso.coef_ != 0)))
print("사용한 max_iter : {}".format(lasso.n_iter_))

훈련 세트의 정확도 : 0.32
테스트 세트의 정확도 : -0.02
사용한 특성의 수 : 117
사용한 max_iter : 48


In [55]:
X_train_columns=X_train.iloc[:,lasso.coef_!=0].columns

In [56]:
X_train_columns

Index(['AGE_B',  'HT_B',  'WT_B', 'SBP_B', 'DBP_B', 'LDL_B',  'TG_B', 'HDL_B',
       'FBS_B', 'GOT_B',
       ...
         14039,   14271,   14311,   14478,   14494,   14809,   14837,   14892,
         15136,   15171],
      dtype='object', length=117)

In [57]:
X_test = X_test[X_train_columns]
X_train = X_train[X_train_columns]

In [58]:
def get_clf_eval(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred) # 오차 행렬 (평가 지표)
    accuracy = accuracy_score(y_test, pred) # 정확도
    precision = precision_score(y_test, pred,average='micro') # 정밀도
    recall = recall_score(y_test, pred,average='micro') # 재현율
    f1 = f1_score(y_test, pred,average='micro') # F1 (정밀도, 재현율 적정 수준 : 1에 가까울 수록 좋음)
    roc_auc = roc_auc_score(y_test, pred_proba,multi_class='ovo') # AUC (1에 가까울 수록 좋음)
    
    print('오차 행렬\n {}'.format(confusion))
    print('정확도 : {0:.4f}, 정밀도 : {1:.4f}, 재현율 : {2:.4f}, F1 : {3:.4f}, AUC: {4:.4f}'
         .format(accuracy, precision, recall, f1, roc_auc))

In [60]:
#logistic Regression
logistic = LogisticRegression()
logistic.fit(X_train, y_train)
pred = logistic.predict(X_test)
pred_proba = logistic.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.5294


In [61]:
#random forest
random = RandomForestClassifier()
random.fit(X_train, y_train)
pred = random.predict(X_test)
pred_proba = random.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.6902


In [62]:
#tree
tree = RandomForestClassifier()
tree.fit(X_train, y_train)
pred = tree.predict(X_test)
pred_proba = tree.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.5288


In [63]:
#adaboost
adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)
pred = adaboost.predict(X_test)
pred_proba = adaboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.4824


In [64]:
#xgboost
xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)
pred = xgboost.predict(X_test)
pred_proba = xgboost.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)

[19:25:47] WARNING: ..\src\learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.3647


In [65]:
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators=400)
lgbm.fit(X_train, y_train)
pred = lgbm.predict(X_test)
pred_proba = lgbm.predict_proba(X_test)[:,1]
get_clf_eval(y_test, pred, pred_proba)


오차 행렬
 [[153   0]
 [  5   0]]
정확도 : 0.9684, 정밀도 : 0.9684, 재현율 : 0.9684, F1 : 0.9684, AUC: 0.4627
